In [2]:
# Import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [3]:
# Import & load the dataset
topcover_data = pd.read_excel("topcover_pilot.xlsx", engine='openpyxl')

# Save as dataframe
topcover_data = pd.DataFrame(topcover_data)

# Check the dataset
print(f"- The Top Cover dataset has {topcover_data.shape[0]} rows and {topcover_data.shape[1]} columns.")
display(topcover_data.head(5))

topcover_data['Mngt_act'].isna().value_counts()
topcover_data['Mngt_act'].value_counts()

- The Top Cover dataset has 1094 rows and 12 columns.


,Pno,Amount,Gender,RG,Cover,SurrVal,Offer/APE,PercCover,CurrAge,Fee_claimed,Take-up ind,Mngt_act
0,1,6444,M,4,48024,0,1.5,0.134183,71.250000,0,0,BCD
1,2,9649,F,7,1000000,0,1.3,0.009649,60.916667,0,0,PIN
2,3,8688,M,8,800001,2250,1.1,0.010860,63.416667,0,0,PIN
3,4,7483,F,4,21713,0,4.7,0.344632,82.916667,0,0,BCD
4,5,8115,M,8,526919,0,1.3,0.015401,60.000000,0,0,BCD


Mngt_act
PIN    632
BCD    251
ACD     81
P5C      7
P5P      7
         5
Name: count, dtype: int64

# Clean the data set

In [4]:
categorical_features = ['Gender', 'RG', 'Fee_claimed', 'Take-up ind', 'Mngt_act']

cleaned_df = topcover_data

# Replace NA values with not reviewed (NR)
cleaned_df['Mngt_act'] = cleaned_df['Mngt_act'].fillna('NR')

# Columns containing string variables
string_variables = ['Mngt_act', 'Gender']

# Removes blanks before and after string variables
for column in string_variables:
    cleaned_df[column] = cleaned_df[column].astype(str).str.strip()
 
# Replace blanks with not reviewed (NR) 
cleaned_df['Mngt_act'] = cleaned_df['Mngt_act'].replace('', 'NR')

# Function to replace missing values in the data
def replace_missing_values(df, ignore_columns, strategy='mean'):
    
    df.replace('N/A', pd.NA, inplace=True)
    
    for column in df.columns:
        if column not in ignore_columns:
            if df[column].isnull().any():
                if strategy == 'mean':
                    replacement_value = df[column].mean()
                elif strategy == 'median':
                    replacement_value = df[column].median()
                else:
                    raise ValueError("Strategy must be either 'mean' or 'median'")
                df[column].fillna(replacement_value, inplace=True)
    return df

cleaned_df = replace_missing_values(topcover_data, categorical_features, strategy='median')




# Feature engineering

In [5]:
# Function to calculate age bands
def age_bands(x):
    if x < 65:
        return 0
    else:
        return 1
    

# Add smoker indicator
cleaned_df['Smoker ind'] = cleaned_df['RG']%2

# Function to calculate socio economic class
def calc_socio_econ_class(x : int) -> int:
    if x <= 2:
        return 1
    elif x <= 4:
        return 2
    elif x <= 6:
        return 3
    elif x <= 8:
        return 4
    
cleaned_df['Socio Economic Class'] = cleaned_df['RG'].apply(calc_socio_econ_class)

# Drop the original rating group variable
cleaned_df = cleaned_df.drop(columns='RG')

# New features

# 1) Surender_val_multiple = Amount/Surrender value
# 2) SurrVal/APE
# 3) SurrVal/Cover
# 4) SurrVal ind
# 5) Age bands

cleaned_df['APE'] = cleaned_df['Amount'] / cleaned_df['Offer/APE']
cleaned_df['SurrVal/APE'] = cleaned_df['SurrVal']/cleaned_df['APE']
cleaned_df['SurrVal/Cover'] = cleaned_df['Amount']/cleaned_df['Cover']
#cleaned_df['Offer/SurrVal'] = np.where(cleaned_df['SurrVal'] != 0, cleaned_df['Amount']/cleaned_df['SurrVal'], np.nan)
cleaned_df['SurrVal ind'] = np.where(cleaned_df['SurrVal'] != 0, 1, 0)
cleaned_df['Post retirement (65) ind'] = cleaned_df['CurrAge'].apply(age_bands)


cleaned_df

,Pno,Amount,Gender,Cover,SurrVal,Offer/APE,PercCover,CurrAge,Fee_claimed,Take-up ind,Mngt_act,Smoker ind,Socio Economic Class,APE,SurrVal/APE,SurrVal/Cover,SurrVal ind,Post retirement (65) ind
0,1,6444,M,48024,0,1.5,0.134183,71.250000,0,0,BCD,0,2,4296.000000,0.000000,0.134183,0,1
1,2,9649,F,1000000,0,1.3,0.009649,60.916667,0,0,PIN,1,4,7422.307692,0.000000,0.009649,0,0
2,3,8688,M,800001,2250,1.1,0.010860,63.416667,0,0,PIN,0,4,7898.181818,0.284876,0.010860,1,0
3,4,7483,F,21713,0,4.7,0.344632,82.916667,0,0,BCD,0,2,1592.127660,0.000000,0.344632,0,1
4,5,8115,M,526919,0,1.3,0.015401,60.000000,0,0,BCD,0,4,6242.307692,0.000000,0.015401,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,1090,21024,M,122000,0,7.3,0.172328,77.500000,0,1,PIN,0,4,2880.000000,0.000000,0.172328,0,1
1090,1091,116604,F,1603567,9398,6.7,0.072715,68.666667,0,1,NR,0,4,17403.582090,0.540004,0.072715,1,1
1091,1092,33361,M,650000,3390,15.5,0.051325,42.333333,1,1,PIN,0,4,2152.322581,1.575043,0.051325,1,0
1092,1093,41296,M,188118,0,15.0,0.219522,77.583333,0,1,PIN,0,4,2753.066667,0.000000,0.219522,0,1


# Create dummies for the categorical variables

In [20]:
features = ['Amount', 'Gender', 'Cover', 'SurrVal', 'Offer/APE',
            'PercCover', 'CurrAge', 'Fee_claimed', 'Take-up ind', 
            'Mngt_act','Socio Economic Class','Smoker ind','SurrVal/APE',
            'SurrVal/Cover','Offer/SurrVal', 'SurrVal ind', 'age_bands']


#split_data_df = pd.get_dummies(cleaned_df[['Gender', 'Mngt_act', 'Socio Economic Class']], columns=['Gender', 'Mngt_act', 'Socio Economic Class'])
cleaned_split_df = pd.get_dummies(cleaned_df, columns=['Gender', 'Mngt_act', 'Socio Economic Class'])

cleaned_split_df.columns.to_list()

columns_to_convert =[
'Gender_F',
'Gender_M',
'Mngt_act_ACD',
'Mngt_act_BCD',
'Mngt_act_NR',
'Mngt_act_P5C',
'Mngt_act_P5P',
'Mngt_act_PIN',
'Socio Economic Class_1',
'Socio Economic Class_2',
'Socio Economic Class_3',
'Socio Economic Class_4']

cleaned_split_df[columns_to_convert] = cleaned_split_df[columns_to_convert].astype(int)
#split_data_df

cleaned_split_df

,Pno,Amount,Cover,SurrVal,Offer/APE,PercCover,CurrAge,Fee_claimed,Take-up ind,Smoker ind,...,Mngt_act_ACD,Mngt_act_BCD,Mngt_act_NR,Mngt_act_P5C,Mngt_act_P5P,Mngt_act_PIN,Socio Economic Class_1,Socio Economic Class_2,Socio Economic Class_3,Socio Economic Class_4
0,1,6444,48024,0,1.5,0.134183,71.250000,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,2,9649,1000000,0,1.3,0.009649,60.916667,0,0,1,...,0,0,0,0,0,1,0,0,0,1
2,3,8688,800001,2250,1.1,0.010860,63.416667,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,4,7483,21713,0,4.7,0.344632,82.916667,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,5,8115,526919,0,1.3,0.015401,60.000000,0,0,0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,1090,21024,122000,0,7.3,0.172328,77.500000,0,1,0,...,0,0,0,0,0,1,0,0,0,1
1090,1091,116604,1603567,9398,6.7,0.072715,68.666667,0,1,0,...,0,0,1,0,0,0,0,0,0,1
1091,1092,33361,650000,3390,15.5,0.051325,42.333333,1,1,0,...,0,0,0,0,0,1,0,0,0,1
1092,1093,41296,188118,0,15.0,0.219522,77.583333,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [21]:
cleaned_split_df.to_excel(r'cleaned data.xlsx', index=False)

# DONT THINK THIS IS NECESSARY, CHECK WITH DEJ

In [22]:
# Identify columns that contain the text "ind" or "_cd" in their names
columns_to_convert = ['Pno', 'Gender', 'RG', 'Fee_claimed', 'Take-up ind', 'Mngt_act']    

# Technically Pno must also be of type object!

# Convert the data type of these columns to object
topcover_data[columns_to_convert] = topcover_data[columns_to_convert].astype('object')